#### ==========================================================
#### 📘 RAG-BASED AUTO GRADER 
#### ==========================================================
#### Requirements:
#### pip install sentence-transformers openai PyPDF2 numpy pandas tiktoken tqdm


In [1]:
# !pip install PyPDF2
# !pip install sentence-transformers openai numpy pandas tqdm genai anthropic 


In [2]:
# ! pip install PyPDF2 python-docx python-pptx pandas sentence-transformers tqdm openai


In [3]:

import os, re, json
import pandas as pd
from tqdm import tqdm
from PyPDF2 import PdfReader
from docx import Document
from pptx import Presentation
from sentence_transformers import SentenceTransformer, util
from openai import OpenAI
import numpy as np


/Users/deva/.pyenv/versions/3.11.9/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ============== CONFIGURATION =============================

DATA_PATH = "./Srinivasan/data/"
CHUNK_SIZE = 550
CHUNK_OVERLAP = 50
EMBED_MODEL = "all-MiniLM-L6-v2"
# LLM_MODEL = "gpt-4o-mini"       
MAX_SCORE = 50



In [5]:
# ! pip install google-generativeai

In [ ]:
# ----------- CONFIG -------------
ACTIVE_LLM_PROVIDER = "openai"  # 🔁 Options: "openai", "gemini", "claude", "llama", "copilot"

embedder = SentenceTransformer(EMBED_MODEL)

In [7]:
# ==========================================================
# 🌐 CLIENT INITIALIZATION
# ==========================================================
if ACTIVE_LLM_PROVIDER == "openai":
    from openai import OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)
    def ask_llm(prompt, model="gpt-4o-mini", temperature=0.2):
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role": "system", "content": "Return STRICT JSON only."},
                      {"role": "user", "content": prompt}],
            temperature=temperature
        )
        return resp.choices[0].message.content.strip()

elif ACTIVE_LLM_PROVIDER == "gemini":
    import google.generativeai as genai
    genai.configure(api_key=GOOGLE_API_KEY)
    def ask_llm(prompt, model="gemini-2.0-flash-lite", temperature=0.2):
        model = genai.GenerativeModel(model)
        resp = model.generate_content(prompt)
        return resp.text.strip()

elif ACTIVE_LLM_PROVIDER == "claude":
    import anthropic
    client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
    def ask_llm(prompt, model="claude-3-5-sonnet-20240620", temperature=0.2):
        msg = client.messages.create(
            model=model,
            max_tokens=1024,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return msg.content[0].text.strip()

elif ACTIVE_LLM_PROVIDER == "llama":
    from huggingface_hub import InferenceClient
    client = InferenceClient(token=HUGGINGFACE_TOKEN)

    def ask_llm(prompt, model="meta-llama/Meta-Llama-3-8B-Instruct", temperature=0.2):
        """
        Uses conversational mode since Llama models are registered under that task.
        """
        messages = [
            {"role": "system", "content": "Return STRICT JSON only."},
            {"role": "user", "content": prompt}
        ]
        resp = client.chat_completion(
            model=model,
            messages=messages,
            max_tokens=1024,
            temperature=temperature,
        )
        return resp["choices"][0]["message"]["content"].strip()


elif ACTIVE_LLM_PROVIDER == "copilot":
    # Example for Azure OpenAI / GitHub Copilot-like API
    import openai
    openai.api_type = "azure"
    openai.api_key = AZURE_API_KEY
    openai.api_base = "https://your-azure-endpoint.openai.azure.com"
    openai.api_version = "2024-03-01-preview"
    def ask_llm(prompt, model="gpt-4", temperature=0.2):
        resp = openai.ChatCompletion.create(
            engine=model,
            messages=[{"role": "system", "content": "Return STRICT JSON only."},
                      {"role": "user", "content": prompt}],
            temperature=temperature
        )
        return resp["choices"][0]["message"]["content"].strip()

else:
    raise ValueError(f"Unsupported provider: {ACTIVE_LLM_PROVIDER}")

print(f"✅ Active LLM provider: {ACTIVE_LLM_PROVIDER}")

✅ Active LLM provider: openai


In [8]:
# ---------- GENERIC TEXT EXTRACTORS ----------
def extract_text_from_pdf(path):
    reader = PdfReader(path)
    return "\n".join([p.extract_text() or "" for p in reader.pages])

def extract_text_from_docx(path):
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])

def extract_text_from_pptx(path):
    prs = Presentation(path)
    text = []
    for s in prs.slides:
        for sh in s.shapes:
            if hasattr(sh, "text"):
                text.append(sh.text)
    return "\n".join(text)

def extract_text_from_csv(path):
    df = pd.read_csv(path)
    return " ".join(df.astype(str).fillna("").values.flatten())

def extract_text_from_txt(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

In [9]:
# ---------- UNIVERSAL LOADER ----------
def extract_text_from_any(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".pdf":
        return extract_text_from_pdf(path)
    elif ext == ".csv":
        return extract_text_from_csv(path)
    elif ext == ".docx":
        return extract_text_from_docx(path)
    elif ext == ".pptx":
        return extract_text_from_pptx(path)
    elif ext == ".txt":
        return extract_text_from_txt(path)
    else:
        print(f"⚠️ Unsupported file type: {path}")
        return ""

In [10]:
# -------------------- CHUNKING -----------------------------
def chunk_text(text, chunk_size=550, overlap=50):
    tokens = re.split(r'\s+', text)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = " ".join(tokens[i:i + chunk_size])
        if chunk.strip():
            chunks.append(chunk)
    return chunks

In [11]:
def load_documents(path, chunk_size=550, overlap=50):
    all_chunks = []

    if os.path.isfile(path):
        # Handle single file
        text = extract_text_from_any(path)
        chunks = chunk_text(text, chunk_size, overlap)
        for c in chunks:
            all_chunks.append({"source": os.path.basename(path), "content": c})
        return all_chunks

    elif os.path.isdir(path):
        # Handle folder with multiple files
        for file in os.listdir(path):
            fpath = os.path.join(path, file)
            if os.path.isfile(fpath):
                text = extract_text_from_any(fpath)
                chunks = chunk_text(text, chunk_size, overlap)
                for c in chunks:
                    all_chunks.append({"source": file, "content": c})
        return all_chunks

    else:
        print(f"⚠️ Invalid path: {path}")
        return []

In [12]:
# -------------------- LOAD RUBRIC -----------------------


def load_rubric_text(path="./data/rubric.csv"):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".csv":
        df = pd.read_csv(path).fillna("")
        return "\n".join([
            f"{r.Criterion} | {r.Weight} | {r.Level} | {r.Description}"
            for r in df.itertuples()
        ])
    return extract_text_from_any(path)

In [13]:
# ==========================================================
# RETRIEVAL
# ==========================================================
notes_docs = load_documents(os.path.join(DATA_PATH, "notes.pdf"))
rubric_text = load_rubric_text(os.path.join(DATA_PATH, "rubric.csv"))
all_texts = [d["content"] for d in notes_docs]
embeddings = embedder.encode(all_texts, convert_to_tensor=True)



In [14]:
# -------------------- RETRIEVAL ----------------------------
def retrieve_context(query, top_k=3):
    q_emb = embedder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(q_emb, embeddings, top_k=top_k)[0]
    return [notes_docs[h["corpus_id"]] for h in hits]

def prepare_indexed_context(retrieved):
    return "\n\n".join([f"[R{i}] {r['content']}" for i, r in enumerate(retrieved, 1)])


In [15]:
# ==========================================================
# COVERAGE CHECK (GROUNDING)
# ==========================================================
def split_sentences(txt):
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', txt) if s.strip()]

def coverage_check(answer, retrieved, sim_threshold=0.5):
    sents = split_sentences(answer)
    if not sents or not retrieved:
        return 0.0, sents
    retr_texts = [r["content"] for r in retrieved]
    retr_emb = embedder.encode(retr_texts, convert_to_tensor=True)
    sent_emb = embedder.encode(sents, convert_to_tensor=True)
    unsupported = []
    for i, e in enumerate(sent_emb):
        max_sim = float(util.cos_sim(e, retr_emb).max().cpu().item())
        if max_sim < sim_threshold:
            unsupported.append(sents[i])
    ratio = 0.0 if not sents else (len(sents)-len(unsupported))/len(sents)
    return ratio, unsupported

In [16]:
# -------------------- PROMPT BUILDER -----------------------
# ==========================================================
# PROMPT (STRICTLY GROUNDED + STRICTNESS CONTROL)
# ==========================================================
def build_prompt(question, answer, context, rubric_text, max_score=50, strictness=3):
    """
    Builds a balanced, notes-grounded grading prompt.
    Includes strictness control (1=very lenient, 5=very strict).
    """

    # 🎚️ Strictness guidance text
    if strictness <= 1:
        tone = (
            "Be lenient. Assume partial understanding even if not perfectly phrased. "
            "Give the benefit of the doubt when the student shows general relevance to the topic."
        )
    elif strictness == 2:
        tone = (
            "Be moderately lenient. Focus on comprehension and intent over precise phrasing. "
            "Minor off-topic points or missing citations should not heavily reduce scores."
        )
    elif strictness == 3:
        tone = (
            "Be balanced and fair. Follow the rubric exactly, rewarding relevance and clarity, "
            "but penalizing factual errors or unsupported claims moderately."
        )
    elif strictness == 4:
        tone = (
            "Be rigorous. Deduct marks for vague or unsupported statements. "
            "Only award high scores for detailed, well-supported, and precise answers."
        )
    else:
        tone = (
            "Be very strict. Grade as a top-tier academic evaluator. "
            "Do not give credit unless the student's response exactly matches information in the notes. "
            "Heavily penalize unsupported or off-topic claims."
        )

    return f"""
You are an academic auto-grader evaluating a student's reflection response.

### GRADING STRICTNESS LEVEL: {strictness}/5
{tone}

### REFERENCE MATERIAL (from course notes)
Use this content as the authoritative source when grading. Only grade ideas that are supported by this material:
{context}

### INSTRUCTOR RUBRIC (from file)
Use this rubric exactly as written. The criteria, weights, and levels define how grading should be done:
{rubric_text}

### QUESTION
{question}

### STUDENT ANSWER
{answer}

---

### YOUR TASK
Grade the student's answer based **only** on the above notes and rubric.

1. **Relevance:** If the student's content does not appear in the notes, treat it as off-topic or unsupported, based on the strictness level.
2. **Per Criterion Evaluation:**
   - Identify the rubric criterion name.
   - Assign a "score" between 0–5.
   - Write a short "comments" paragraph (1–2 sentences) specific to the topic, explaining what was good or missing.
   - Avoid generic phrases like “good job” or “needs more detail.” Instead, reference the actual content (e.g., Dakota, land, sacred power, geography, identity, etc.).
3. **Overall Feedback:**
   - Compute the overall grade according to the rubric’s weights and levels.
   - Provide a short "feedback_summary" (2–3 sentences) summarizing performance.
   - Mention specific areas of strength or improvement related to the question.
4. **Unsupported Content:**
   - List exact sentences or ideas from the student’s answer that are **not supported by the notes**.
5. **Correct Answer Retrieval:**
   - From the provided notes, find and quote or paraphrase the most relevant passage that represents the correct answer. 
   - Include its source name or page number if visible.

---

### OUTPUT FORMAT (STRICT JSON ONLY)
Return only a valid JSON object in this structure:

{{
  "criteria": [
    {{
      "criterion": "<criterion name from rubric>",
      "score": 4,
      "comments": "Shows clear understanding of the Dakota concept of sacred geography, but lacks examples from the notes."
    }},
    {{
      "criterion": "<criterion name from rubric>",
      "score": 5,
      "comments": "Well written, clear, and supported by the course material."
    }}
  ],
  "unsupported_claims": [
    "The Dakota people worshipped in temples."
  ],
  "final_score": 0,
  "max_score": {max_score},
  "feedback_summary": "Good comprehension and structure, though some claims are not supported by notes.",
  "correct_answer": {{
    "source": "notes.pdf page 3",
    "content": "The 'sacred power of place' refers to the Dakota belief that land itself is sacred and embodies memory and identity..."
  }}
}}

### RULES
- Output **JSON only** — no markdown, no explanations.
- Follow the **strictness level** when deciding leniency or harshness.
- Always include `"criterion"`, `"score"`, `"comments"` for each rubric section.
- Always provide `"correct_answer"` with source and content.
"""


In [17]:
# -------------------- SAFE JSON PARSER -----------------------

import json, re

def safe_json_parse(raw_output: str):
    """
    Safely parse possibly malformed JSON output from an LLM.
    Tries multiple cleaning strategies automatically.
    """
    if not raw_output or not raw_output.strip():
        raise ValueError("Empty response from model — no JSON returned.")

    candidates = [raw_output]

    # remove markdown fences
    candidates.append(re.sub(r"```(json)?", "", raw_output).strip())

    # replace single quotes with double quotes cautiously
    candidates.append(re.sub(r"'", '"', raw_output))

    # remove trailing commas
    candidates.append(re.sub(r",\s*([}\]])", r"\1", raw_output))

    for text in candidates:
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            continue

    # final fallback: extract possible JSON substring
    match = re.search(r"\{.*\}", raw_output, re.S)
    if match:
        try:
            return json.loads(match.group(0))
        except json.JSONDecodeError:
            pass

    print("⚠️ Could not parse model JSON output:\n", raw_output[:600])
    raise json.JSONDecodeError("LLM output not valid JSON", raw_output, 0)


In [ ]:
def grade_answer(question, answer, max_score=50, top_k=3, sim_threshold=0.5, strictness=3):
    """
    Notes-grounded grading (balanced):
    - Grades student answer using rubric + note context.
    - Lists unsupported claims.
    - Returns the best-matched 'correct answer' snippet from notes.
    - Keeps JSON clean and human-readable.
    """

    # --- Retrieve note chunks related to question ---
    query = f"{question} {answer}"
    retrieved = retrieve_context(query, top_k=top_k)
    context = prepare_indexed_context(retrieved)

    # --- Build prompt for the LLM ---
    prompt = build_prompt(
        question=question,
        answer=answer,
        context=context,
        rubric_text=rubric_text,
        max_score=max_score,
        strictness = strictness
    )

    # --- Ask the LLM to grade ---
    raw_output = ask_llm(prompt)

    print("\n🧠 Raw LLM Output:\n", raw_output[:1000], "\n")

    # --- Parse JSON safely ---
    data = safe_json_parse(raw_output)

    # --- Normalize per-criterion fields ---
    for i, c in enumerate(data.get("criteria", []), 1):
        if "criterion" not in c:
            c["criterion"] = (
                c.get("criteria_name")
                or c.get("name")
                or f"Criterion {i}"
            )
        try:
            c["score"] = float(c.get("score", 0))
        except:
            c["score"] = 0.0
        c["score"] = max(0.0, min(5.0, c["score"]))
        if "comments" not in c:
            c["comments"] = ""

    # --- Parse rubric weights dynamically ---
    weights = {}
    for line in rubric_text.splitlines():
        parts = [p.strip() for p in line.split("|")]
        if len(parts) >= 3:
            crit = parts[0].split("(")[0].strip()
            try:
                w = float(parts[1])
                if 0 <= w <= 1.0:
                    weights[crit] = w
            except:
                pass
    if not weights and data.get("criteria"):
        eq = 1.0 / len(data["criteria"])
        for c in data["criteria"]:
            base = c["criterion"].split("(")[0].strip()
            weights[base] = eq

    # --- Compute weighted total (no strict penalty) ---
    total_weight, weighted_sum = 0.0, 0.0
    for c in data.get("criteria", []):
        base = c["criterion"].split("(")[0].strip()
        w = weights.get(base, 1.0 / len(data["criteria"]))
        total_weight += w
        weighted_sum += (c["score"] / 5.0) * w

    final_score = round((weighted_sum / total_weight) * max_score, 2) if total_weight else 0.0

    # --- Identify unsupported claims using embeddings ---
    unsupported = []
    answer_sents = split_sentences(answer)
    retr_texts = [r["content"] for r in retrieved]
    retr_emb = embedder.encode(retr_texts, convert_to_tensor=True)
    sent_emb = embedder.encode(answer_sents, convert_to_tensor=True)
    for i, e in enumerate(sent_emb):
        max_sim = float(util.cos_sim(e, retr_emb).max().cpu().item())
        if max_sim < sim_threshold:
            unsupported.append(answer_sents[i])

    # --- Retrieve 'correct answer' snippet from notes ---
    q_emb = embedder.encode(question, convert_to_tensor=True)
    sims = util.cos_sim(q_emb, retr_emb)[0].cpu().tolist()
    best_idx = int(max(range(len(sims)), key=lambda i: sims[i]))
    best_snippet = retrieved[best_idx]["content"].strip()
    source_name = retrieved[best_idx]["source"]

    # --- Compose final clean JSON output ---
    output = {
        "criteria": [
            {
                "criterion": c["criterion"],
                "score": c["score"],
                "comments": c["comments"],
            }
            for c in data.get("criteria", [])
        ],
        "unsupported_claims": unsupported,
        "final_score": min(max_score, round(final_score, 2)),
        "max_score": max_score,
        "feedback_summary": data.get(
            "feedback_summary",
            "Evaluation based on provided notes and rubric. Unsupported statements noted."
        ),
        "correct_answer": {
            "source": source_name,
            "content": best_snippet[:700] + ("..." if len(best_snippet) > 700 else "")
        }
    }

    return output


In [30]:
question = "Why is land considered central to the religion and cultural identity of the Dakota people?"
answer = "Land is central to Dakota religion and cultural identity because it is seen as the place of human origin and the center of the world, where sacred stories, ceremonies, and generational memory connect people to their ancestors and the spiritual realm. Loss of land disrupts this connection, making geography inseparable from their religious and cultural life."
result = grade_answer(question, answer, max_score=50, top_k=3, sim_threshold=0.5, strictness=1)
print(json.dumps(result, indent=2))



🧠 Raw LLM Output:
 {
  "criteria": [
    {
      "criterion": "Critical Analysis (understanding of course materials)",
      "score": 4,
      "comments": "The response demonstrates a solid understanding of the importance of land to Dakota identity and religion, particularly in terms of ancestral connections and sacred stories."
    },
    {
      "criterion": "Academic and Scholarly Presentation",
      "score": 5,
      "comments": "The reflection is well-articulated and free from errors, clearly conveying the ideas related to Dakota cultural identity."
    },
    {
      "criterion": "Portrays Insight (follows instructional questions)",
      "score": 4,
      "comments": "The answer engages well with the question, highlighting the inseparability of land from Dakota religion and culture, but could benefit from more specific examples from the course material."
    }
  ],
  "unsupported_claims": [],
  "final_score": 42,
  "max_score": 50,
  "feedback_summary": "The student shows a st

In [20]:
result = grade_answer(question, answer, max_score=50, top_k=3, sim_threshold=0.5, strictness=2)
print(json.dumps(result, indent=2))


🧠 Raw LLM Output:
 {
  "criteria": [
    {
      "criterion": "Critical Analysis (understanding of course materials)",
      "score": 4,
      "comments": "The response demonstrates a solid understanding of the significance of land in Dakota culture and religion, particularly its connection to ancestry and spirituality."
    },
    {
      "criterion": "Academic and Scholarly Presentation",
      "score": 5,
      "comments": "The answer is well-articulated and free from errors, clearly conveying the importance of land to the Dakota people."
    },
    {
      "criterion": "Portrays Insight (follows instructional questions)",
      "score": 4,
      "comments": "The response engages well with the question, though it could benefit from more specific examples from the course materials."
    }
  ],
  "unsupported_claims": [],
  "final_score": 42,
  "max_score": 50,
  "feedback_summary": "The student shows a good understanding of the relationship between land and Dakota identity, with cle

In [21]:
result = grade_answer(question, answer, max_score=50, top_k=3, sim_threshold=0.5, strictness=3)
print(json.dumps(result, indent=2))


🧠 Raw LLM Output:
 {
  "criteria": [
    {
      "criterion": "Critical Analysis (understanding of course materials)",
      "score": 4,
      "comments": "The response demonstrates a solid understanding of the importance of land in Dakota culture and religion, but it could benefit from more specific examples from the course materials."
    },
    {
      "criterion": "Academic and Scholarly Presentation",
      "score": 5,
      "comments": "The answer is clearly articulated and free from errors, following proper formatting and structure."
    },
    {
      "criterion": "Portrays Insight (follows instructional questions)",
      "score": 4,
      "comments": "The student engages well with the question and provides relevant insights, though some aspects could be elaborated further."
    }
  ],
  "unsupported_claims": [],
  "final_score": 42,
  "max_score": 50,
  "feedback_summary": "The response shows a good understanding of the centrality of land to Dakota identity and religion, tho

In [22]:
result = grade_answer(question, answer, max_score=50, top_k=3, sim_threshold=0.5, strictness=4)
print(json.dumps(result, indent=2))


🧠 Raw LLM Output:
 {
  "criteria": [
    {
      "criterion": "Critical Analysis (understanding of course materials)",
      "score": 4,
      "comments": "The response demonstrates a solid understanding of the significance of land to Dakota identity, but it could benefit from more specific examples from the course materials."
    },
    {
      "criterion": "Academic and Scholarly Presentation",
      "score": 5,
      "comments": "The answer is well-articulated and free from errors, clearly presenting the ideas in a coherent manner."
    },
    {
      "criterion": "Portrays Insight (follows instructional questions)",
      "score": 4,
      "comments": "The student engages with the question effectively, but the response could be enhanced by deeper exploration of the connections between land and Dakota spirituality."
    }
  ],
  "unsupported_claims": [],
  "final_score": 42,
  "max_score": 50,
  "feedback_summary": "The student shows a good understanding of the importance of land t

In [23]:
result = grade_answer(question, answer, max_score=50, top_k=3, sim_threshold=0.5, strictness=5)
print(json.dumps(result, indent=2))


🧠 Raw LLM Output:
 {
  "criteria": [
    {
      "criterion": "Critical Analysis (understanding of course materials)",
      "score": 4,
      "comments": "The response demonstrates a solid understanding of the importance of land to Dakota identity and religion, but it lacks specific examples from the notes that could strengthen the argument."
    },
    {
      "criterion": "Academic and Scholarly Presentation",
      "score": 5,
      "comments": "The writing is clear and well-structured, with no grammatical errors, and follows the prompt effectively."
    },
    {
      "criterion": "Portrays Insight (follows instructional questions)",
      "score": 4,
      "comments": "The answer engages with the question and reflects on the connection between land and Dakota identity, but could benefit from deeper exploration of the cultural implications."
    }
  ],
  "unsupported_claims": [],
  "final_score": 42,
  "max_score": 50,
  "feedback_summary": "The response shows a good understandin

In [24]:
# question = "What is meant by 'the sacred power of place' in Dakota religion?"
# answer = "The 'sacred power of place' refers to the belief that land itself holds spiritual memory and identity. For the Dakota, geography is religion: rivers, rocks, and villages embody ancestral stories. Losing land means losing part of the sacred self."
# result = grade_answer(question, answer, max_score=10)
# print(json.dumps(result, indent=2))


In [25]:
# partial answer 

# question = "Why is land considered central to the religion and cultural identity of the Dakota people?"
# partial_answer = "It means that Dakota people see land as important for their religion and culture, and losing it affects them."
# result = grade_answer(question, partial_answer, max_score=50, top_k=3, sim_threshold=0.5, strictness=5)
# print(json.dumps(result, indent=2))


In [26]:
question = "What is meant by 'the sacred power of place' in Dakota religion?"
wrong_answer = "It means that Dakota people see land as important for their religion and culture,"
result = grade_answer(question, wrong_answer, max_score=10,  top_k=3, sim_threshold=0.5, strictness=3)
print(json.dumps(result, indent=2))


🧠 Raw LLM Output:
 {
  "criteria": [
    {
      "criterion": "Critical Analysis (understanding of course materials)",
      "score": 2,
      "comments": "The response identifies the importance of land in Dakota religion but lacks depth and specific connections to the concept of 'sacred power of place' as described in the course materials."
    },
    {
      "criterion": "Academic and Scholarly Presentation",
      "score": 4,
      "comments": "The answer is clear and mostly free from errors, but it could benefit from more detailed explanations and examples from the course materials."
    },
    {
      "criterion": "Portrays Insight (follows instructional questions)",
      "score": 2,
      "comments": "The response addresses the question but does so superficially without engaging deeply with the course material or providing substantial insight."
    }
  ],
  "unsupported_claims": [],
  "final_score": 4,
  "max_score": 15,
  "feedback_summary": "The response demonstrates a basic 

In [ ]:
# question = "How can we correlate applied history with our present?"
# check_answer = "Applied history uses these patterns to analyze current issues and assess the potential consequences of policy decisions. By understanding historical context, we gain perspective to navigate contemporary challenges and inform future planning"
# result = grade_answer(question, check_answer, max_score=10)
# print(json.dumps(result, indent=2))